# Evaluating the Models

## This Juypter Notebook was created to easily load the models and set/change the directories to test the models, all in one place.

In [ ]:
import os
from database import Database
from models.altered_xception import AlteredXception
from models.faster_r_cnn import FasterRCNN
from models.rmac_model import RMACModel
from utils import grab_images_and_paths, grab_all_image_paths
from models.query import query_image
from utils import EVAL_PATH

## Loading The Models and Data
> Note: if you get any error surrounding a missing layer, a simple restart of the kernel should fix it.

In [ ]:
# NOTE: if the RMAC model needs to get descriptions of many different square regions, it will take a long time to load/initialize (roughly a minute).
ax_model = AlteredXception()
rmac_model = RMACModel(ax_model.model.get_layer("conv2d_3").output_shape[1:], 3)
rcnn_model = FasterRCNN()

In [ ]:
# Choose what models to evaluate, by including or discluding them from this array
models = [ax_model, rmac_model, rcnn_model]

In [ ]:
# test_dir is the directory where our "database" of test images/frames and query images should exist. 
test_dir = os.path.join(EVAL_PATH, "test_dogs")
frame_paths, frames = grab_images_and_paths(os.path.join(test_dir, "frames"), num_images=1) # Number of images doesn't matter unless we want to confirm the database is complete.
query_image_paths = grab_all_image_paths(os.path.join(test_dir, "query_images"))
query_image_paths

In [ ]:
# Get the YouTube video "ground truth" from the IDs we picked, stored in a .txt file
ground_truth_ids = []
ground_truth_file_path = os.path.join(test_dir, "ground_truth.txt")
with open(ground_truth_file_path, "r") as ground_truth:
    for line in ground_truth.readlines():
        youtube_ground_truth_id = line.split("/")[-1].split(".")[0]
        ground_truth_ids.append(youtube_ground_truth_id)
ground_truth_ids

In [ ]:
# Initialize models to query from the evaluation database
database = Database(os.path.join(test_dir, "predictions")) # Small DB
# database = Database(DATABASE_PATH) # Large DB 
for model in models:
    model.database = database

In [ ]:
# Gathering predictions for each of the frames in the dataset, for each model
if not os.path.exists(os.path.join(test_dir, "predictions")):
    # Make the folder to store our representations
    os.mkdir(os.path.join(test_dir, "predictions"))
for model in models:
    # Check that we've downloaded all the frames for each model. If not, gather and store them.
    if (isinstance(model, AlteredXception)) and len(database.prediction_image_paths) < len(frame_paths):
        model_predictions = model.predict_images(frames)
        database.store_predictions(model_predictions, frame_paths)
    if (isinstance(model, RMACModel)) and len(database.rmac_prediction_image_paths) < len(frame_paths):
        model_predictions = model.predict_images(frames)
        database.store_rmac_predictions(model_predictions, frame_paths)
    if (isinstance(model, FasterRCNN)) and len(database.object_predictions) < len(frame_paths):
        every_10_frame_paths = frame_paths[0::10]
        model_predictions = model.predict_image_paths(every_10_frame_paths)
        database.store_object_data(model_predictions, every_10_frame_paths)

In [ ]:
def post_process_predictions(predictions):
    ''' Post processes the predictions we receive to make comparisons/results easier to read. '''
    model_preds = {}
    for model in predictions:
        video_id_preds = []
        for video_link in predictions[model].keys():
            video_id = video_link.split("/")[-1] # We only want the last part of the path
            video_id_preds.append(video_id)
        model_preds[model] = video_id_preds
    return model_preds

## Get The Results From the Model

In [ ]:
results = {}
for query_image_path in query_image_paths:
    results[query_image_path] = {}
    results[query_image_path] = post_process_predictions(query_image(query_image_path, models, len(ground_truth_ids)))
results

## Get the final evaluation results!

In [ ]:
# Compute the evaluation results, computing the number of matches between my ground truth and the models' predictions

matches = {}
for query_image_path in query_image_paths:
    curr_results = results[query_image_path]
    matches[query_image_path] = {}
    for model in results[query_image_path]:
        num_matches = []
        for index, id in enumerate(ground_truth_ids):
            if id in results[query_image_path][model]:
                num_matches.append(index+1)
        matches[query_image_path][model] = num_matches
matches